# Converting a torch model to ONNX using Default PyToch export
This notebook shows how we can transform PyTorch models into the ONNX format so we can use our model in the browser.

First, imagine we load the model from somewhere. Below, we use the potsawee model, but in the future we will use our own trained model.

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("potsawee/t5-large-generation-squad-QuestionAnswer")

Next we export the model to onnx. There are several things we have to do to make this work.
1. We must generate sample inputs for the model using the Tokenizer. According to [the documentation](https://pytorch.org/docs/stable/onnx.html), this is done because to convert the model to the ONNX format, the model needs to be run once with its proper input.
2. We should also provide the input and output names for columns.

In [1]:
tokenizer = AutoTokenizer.from_pretrained("potsawee/t5-large-generation-squad-QuestionAnswer")

sample_text = "In the beginning, God created the heavens and the earth."
inputs = tokenizer(sample_text, return_tensors="pt")
outputs = model.generate(**inputs)
question_answer = tokenizer.decode(outputs[0], skip_special_tokens=False)
decoder_input_ids = outputs[0]

onnx_path = "onnxDefault/potsawee.onnx"
input_names = ["input_ids", "attention_mask"]
output_names = ["output"]
dynamic_axes = {"input_ids": {0: "batch_size", 1: "seq_len"}, "attention_mask": {0: "batch_size", 1: "seq_len"}, "output": {0: "batch_size", 1: "seq_len"}}

NameError: name 'AutoTokenizer' is not defined

Next we use the `torch.onnx.export` function to export the model.

In [8]:
print("inputs:", inputs)
print("vals:", inputs.values())
print("outputs:", outputs)
print("question_answer:", question_answer)
print("decoder_input_ids:", decoder_input_ids)

inputs: {'input_ids': tensor([[  86,    8, 1849,    6,  601,  990,    8, 9922,    7,   11,    8, 3596,
            5,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
vals: dict_values([tensor([[  86,    8, 1849,    6,  601,  990,    8, 9922,    7,   11,    8, 3596,
            5,    1]]), tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])])
outputs: tensor([[    0,  2645,   990,     8,  9922,     7,    11,     8,  3596,    58,
         32100,   601,     1]])
question_answer: <pad> Who created the heavens and the earth?<sep> God</s>
decoder_input_ids: tensor([    0,  2645,   990,     8,  9922,     7,    11,     8,  3596,    58,
        32100,   601,     1])


In [9]:
torch.onnx.export(model, args=tuple(inputs.values()) + (decoder_input_ids,), f=onnx_path, input_names=input_names, output_names=output_names, dynamic_axes=dynamic_axes)

============== Diagnostic Run torch.onnx.export version 2.0.0+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



ValueError: not enough values to unpack (expected 2, got 1)

And that should be it! There should be a `potsawee.onnx` file in the `onnxModel` directory now. You can also see hundreds of `onnx_*` files that are generated. These are part of the model. Don't erase them.